In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2
import seaborn as sns

In [4]:
train_files = os.listdir(r"C:\Users\meetd\OneDrive\Desktop\Datasets\dog-breed\train")
test_files = os.listdir(r"C:\Users\meetd\OneDrive\Desktop\Datasets\dog-breed\test")
print("Train size = ", len(train_files))
print("Test size = ", len(test_files))

Train size =  10222
Test size =  10357


In [5]:
labels = pd.read_csv(r'C:\Users\meetd\OneDrive\Desktop\Datasets\dog-breed\labels.csv')
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [6]:
dog_breeds = sorted(labels['breed'].unique())
num_classes = len(dog_breeds)
print(num_classes)

120


In [7]:
dog_breeds

['affenpinscher',
 'afghan_hound',
 'african_hunting_dog',
 'airedale',
 'american_staffordshire_terrier',
 'appenzeller',
 'australian_terrier',
 'basenji',
 'basset',
 'beagle',
 'bedlington_terrier',
 'bernese_mountain_dog',
 'black-and-tan_coonhound',
 'blenheim_spaniel',
 'bloodhound',
 'bluetick',
 'border_collie',
 'border_terrier',
 'borzoi',
 'boston_bull',
 'bouvier_des_flandres',
 'boxer',
 'brabancon_griffon',
 'briard',
 'brittany_spaniel',
 'bull_mastiff',
 'cairn',
 'cardigan',
 'chesapeake_bay_retriever',
 'chihuahua',
 'chow',
 'clumber',
 'cocker_spaniel',
 'collie',
 'curly-coated_retriever',
 'dandie_dinmont',
 'dhole',
 'dingo',
 'doberman',
 'english_foxhound',
 'english_setter',
 'english_springer',
 'entlebucher',
 'eskimo_dog',
 'flat-coated_retriever',
 'french_bulldog',
 'german_shepherd',
 'german_short-haired_pointer',
 'giant_schnauzer',
 'golden_retriever',
 'gordon_setter',
 'great_dane',
 'great_pyrenees',
 'greater_swiss_mountain_dog',
 'groenendael',


In [8]:
classes_to_num = dict(zip(dog_breeds,range(num_classes)))

In [13]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
def images_to_array(data_dir, labels, image_size):
    image_names = labels['id']
    image_labels = labels['breed']
    data_size = len(image_names)
    x = np.zeros([data_size,image_size[0],image_size[1],image_size[2]],dtype=np.uint8)
    y = np.zeros([data_size,1],dtype = np.uint8)
    for i in range(data_size):
        img_name = image_names[i]
        img_dir = os.path.join(data_dir, img_name+'.jpg')
        img_pixels = load_img(img_dir,target_size=image_size)
        x[i] = img_pixels
        y[i] = classes_to_num[image_labels[i]]
    y = to_categorical(y)
    ind = np.random.permutation(data_size)
    x = x[ind]
    y = y[ind]
    print('Ouptut Data Size: ', x.shape)
    print('Ouptut Label Size: ', y.shape)
    return x, y

In [14]:
img_size = (299,299,3)
x, y = images_to_array(r"C:\Users\meetd\OneDrive\Desktop\Datasets\dog-breed\train", labels, img_size)

Ouptut Data Size:  (10222, 299, 299, 3)
Ouptut Label Size:  (10222, 120)


In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D, Lambda, Dropout, InputLayer, Input

def get_features(model_name, data_preprocessor, input_size, data):
    input_layer = Input(input_size)
    preprocessor = Lambda(data_preprocessor)(input_layer)
    base_model = model_name(weights='imagenet', include_top=False,
                            input_shape=input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    feature_maps = feature_extractor.predict(data, batch_size=32, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

In [18]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
inception_preprocessor = preprocess_input
inception_features = get_features(InceptionV3,
                                  inception_preprocessor,
                                  img_size, x)

320/320 [==============================] - 1072s 3s/step
Feature maps shape:  (10222, 2048)


In [19]:
from tensorflow.keras.applications.xception import Xception, preprocess_input
xception_preprocessor = preprocess_input
xception_features = get_features(Xception,
                                 xception_preprocessor,
                                 img_size, x)

320/320 [==============================] - 1709s 5s/step
Feature maps shape:  (10222, 2048)


In [21]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
inc_resnet_preprocessor = preprocess_input
inc_resnet_features = get_features(InceptionResNetV2,
                                   inc_resnet_preprocessor,
                                   img_size, x)

320/320 [==============================] - 2157s 7s/step
Feature maps shape:  (10222, 1536)


In [22]:
final_features = np.concatenate([inception_features,
                                 xception_features,
                                 inc_resnet_features,], axis=-1)
print('Final feature maps shape', final_features.shape)

Final feature maps shape (10222, 5632)


In [23]:
del x

In [24]:
from tensorflow.keras.callbacks import EarlyStopping
EarlyStop_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
my_callback=[EarlyStop_callback]

In [25]:
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(InputLayer(final_features.shape[1:]))
model.add(Dropout(0.7))
model.add(Dense(120,activation='softmax'))

In [26]:
model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [27]:
history = model.fit(final_features,
                  y,
                  batch_size=32,
                  epochs=50,
                  validation_split=0.1,
                  callbacks=my_callback)

Epoch 1/50
288/288 [==============================] - 3s 8ms/step - loss: 0.8543 - accuracy: 0.7976 - val_loss: 0.3158 - val_accuracy: 0.9052
Epoch 2/50
288/288 [==============================] - 2s 7ms/step - loss: 0.2714 - accuracy: 0.9153 - val_loss: 0.3237 - val_accuracy: 0.9013
Epoch 3/50
288/288 [==============================] - 2s 7ms/step - loss: 0.2134 - accuracy: 0.9310 - val_loss: 0.3356 - val_accuracy: 0.9110
Epoch 4/50
288/288 [==============================] - 2s 6ms/step - loss: 0.1810 - accuracy: 0.9371 - val_loss: 0.3270 - val_accuracy: 0.9042
Epoch 5/50
288/288 [==============================] - 2s 6ms/step - loss: 0.1577 - accuracy: 0.9459 - val_loss: 0.3494 - val_accuracy: 0.9071
Epoch 6/50
288/288 [==============================] - 2s 6ms/step - loss: 0.1420 - accuracy: 0.9499 - val_loss: 0.3227 - val_accuracy: 0.9071
Epoch 7/50
288/288 [==============================] - 2s 6ms/step - loss: 0.1177 - accuracy: 0.9586 - val_loss: 0.3599 - val_accuracy: 0.9042
Epoch 

In [28]:
def images_to_array2(data_dir,df, img_size):
    images_names = df['id']
    data_size = len(images_names)
    X = np.zeros([data_size, img_size[0], img_size[1], 3], dtype=np.uint8)
    
    for i in range(data_size):
        image_name = images_names[i]
        img_dir = os.path.join(data_dir, image_name+'.jpg')
        img_pixels = load_img(img_dir, target_size=img_size)
        X[i] = img_pixels
        
    print('Ouptut Data Size: ', X.shape)
    return X

In [29]:
sample_df = pd.read_csv(r'C:\Users\meetd\OneDrive\Desktop\Datasets\dog-breed\sample_submission.csv')

In [31]:
test_data = images_to_array2(r'C:\Users\meetd\OneDrive\Desktop\Datasets\dog-breed\test', sample_df, img_size)

Ouptut Data Size:  (10357, 299, 299, 3)


In [32]:
inception_features = get_features(InceptionV3, inception_preprocessor, img_size, test_data)

324/324 [==============================] - 938s 3s/step
Feature maps shape:  (10357, 2048)


In [33]:
xception_features = get_features(Xception, xception_preprocessor, img_size, test_data)

324/324 [==============================] - 1472s 5s/step
Feature maps shape:  (10357, 2048)


In [34]:
inc_resnet_features = get_features(InceptionResNetV2, inc_resnet_preprocessor, img_size, test_data)

324/324 [==============================] - 2012s 6s/step
Feature maps shape:  (10357, 1536)


In [35]:
test_features = np.concatenate([inception_features,
                                 xception_features,
                                 inc_resnet_features],axis=-1)
print('Final feature maps shape', test_features.shape)

Final feature maps shape (10357, 5632)


In [36]:
del test_data

In [37]:
y_pred = model.predict(test_features, batch_size=32)

In [40]:
for breed in dog_breeds:
    sample_df[breed] = y_pred[:,classes_to_num[breed]]
sample_df.to_csv(r'C:\Users\meetd\OneDrive\Desktop\Datasets\my_submission.csv', index=None)

In [41]:
sample_df.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,6.908525e-06,0.000003,0.000001,7.010412e-07,0.000002,0.000002,1.128423e-06,0.000001,0.000003,...,0.000005,1.195549e-06,0.000003,2.443184e-07,0.000003,4.982917e-07,7.224523e-07,9.541774e-07,7.531496e-07,0.000006
1,00102ee9d8eb90812350685311fe5890,4.867068e-06,0.000003,0.000003,4.193643e-06,0.000041,0.000020,8.194342e-06,0.000037,0.000017,...,0.000015,3.497116e-06,0.000013,2.242612e-05,0.000006,6.081425e-07,3.258623e-04,1.020651e-05,5.965836e-07,0.000007
2,0012a730dfa437f5f3613fb75efcd4ce,5.127299e-07,0.000056,0.000007,2.251797e-06,0.000001,0.000004,9.838795e-07,0.000001,0.000037,...,0.000003,5.461209e-07,0.000002,7.817844e-06,0.000008,2.100909e-05,4.470284e-07,2.775327e-05,3.682630e-07,0.000003
3,001510bc8570bbeee98c8d80c8a95ec1,2.428399e-05,0.000002,0.000002,5.901118e-07,0.000028,0.000012,1.454514e-06,0.000013,0.000013,...,0.000030,4.170742e-07,0.000003,4.036266e-07,0.000002,1.063759e-07,8.210928e-07,6.545527e-06,1.122516e-08,0.000009
4,001a5f3114548acdefa3d4da05474c2e,1.039411e-02,0.000089,0.000039,2.407017e-06,0.000127,0.000074,3.360310e-05,0.000094,0.000225,...,0.000119,2.909066e-05,0.000030,3.035041e-05,0.000014,4.715371e-06,1.583395e-05,3.116720e-04,8.699520e-06,0.001548
